# Billboard Hot 100 Song Scraper
Pulls together a list of songs that have been on the Billboard Hot 100 Charts through a Python API (Documentation found here https://github.com/guoguo12/billboard-charts).

## 1. Import necessary libraries

In [1]:
import billboard
import numpy as np
import pandas as pd
import csv
from datetime import datetime  
from datetime import timedelta  



'2018-05-05'

## 2. Create a list of weeks starting from 1/2/10 to present, so that we can call the API in a huge "FOR" loop

In [2]:
start_date='2010-01-02'
y=datetime.strptime(start_date, '%Y-%m-%d').date()
billboard_dates=[start_date]
for weeks in range (1,500):
    y=y + timedelta(days=7)
    
    z=y.strftime("%Y-%m-%d")
    billboard_dates += [z]
    if datetime.today().date()-timedelta(days=7) < y:
        break
        

## 3. Loop through the Charts by week, and store all 100 songs that appear during that week in a list of lists.

In [3]:
chart_rankings=[]
for dates in billboard_dates:
    chart = billboard.ChartData('hot-100',date=dates, fetch=True, timeout=25)
    for x in range(100):
        y=[chart.date,chart[x].title,chart[x].artist,chart[x].rank, chart[x].peakPos,chart[x].lastPos,chart[x].weeks,chart[x].isNew]
        chart_rankings+=[y]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 4. Preprocessing DataFrame so that the Spotify API Search Function can easily digest data.

In [12]:
bbdf= pd.DataFrame(chart_rankings)
bbdf.columns=['date','track','artist','current_position','peak_position','last_position','weeks_on_chart','new?']

In [15]:
#Spotify API doesn't handle "Featuring" that well, we can replace that and similar words with ","
bbdf.artist=bbdf.artist.str.replace(' Featuring ',', ')
bbdf.artist=bbdf.artist.str.replace(' featuring ',', ')
bbdf.artist=bbdf.artist.str.replace('feat. ',', ')
bbdf.artist=bbdf.artist.str.replace(' x ',', ')
bbdf.artist=bbdf.artist.str.replace(' X ',', ')
bbdf.artist=bbdf.artist.str.replace(', ',', ')
bbdf.artist=bbdf.artist.str.replace(' & ',', ')
bbdf.artist=bbdf.artist.str.replace(' Or ',', ')
#Jay-Z has no songs on Spotify, #ThanksTidal
bbdf=bbdf[~bbdf.artist.str.contains('jay-z',case=False)] 
#DJ Khaled collaborates with way too many people and the Spotify API will break
bbdf.artist=bbdf.artist.str.replace('DJ Khaled / Nicki Minaj, Chris Brown, August Alsina, Jeremih, Future & Rick Ross','DJ Khaled')
bbdf.artist=bbdf.artist.str.replace(' / Swae Lee / Rae Sremmurd','')

/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


## 5. Loading the list of songs to search to the Spotify API

In [21]:
bbdf['combined']=bbdf.track+' - '+bbdf.artist

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
new=list(bbdf.combined)
new=set(new)

In [31]:
#unm, clientid/secret redacted. I did not use a config file for this, but going forward will have a separate config file.

import spotipy
import requests
from spotipy import util
import time
unm = "private"
scope = "streaming"

token = util.prompt_for_user_token(
    unm,
    scope,
    client_id='private',
    client_secret='private',
    redirect_uri='http://localhost:8889/tree')
sp = spotipy.Spotify(auth=token)

Populating the interactive namespace from numpy and matplotlib


In [32]:
#attempts to pull the unique "uri" for a given song + artist combination. If this does not exist, pass.
#There is roughly 5-10% of missed songs due to the way the combinations are written
uris=[]
for track in new:
    try:
        entry= sp.search(q=track,type='track',market='us')['tracks']['items'][0]
        uris += [[entry['uri'],entry['album']['release_date']]]
        print(track+' complete')
    except IndexError:
        print('out of range')

It's Not Unusual - Glee Cast complete
2U - David Guetta, Justin Bieber complete
X - 21 Savage, Metro Boomin, Future complete
Don't Judge Me - Chris Brown complete
Friday - Rebecca Black complete
I Lived - OneRepublic complete
She Will - Lil Wayne, Drake complete
Tennis Court - Lorde complete
No Problem - Chance The Rapper, Lil Wayne, 2 Chainz complete
Beauty And The Beast - Ariana Grande, John Legend complete
This Is Gospel - Panic! At The Disco complete
Snitches Ain't... - YG, Tyga, Snoop Dogg, Nipsey Hussle complete
Fix A Drink - Chris Janson complete
X - Lil Uzi Vert complete
No BS - Chris Brown complete
History - One Direction complete
Carolina - Parmalee complete
Me, Myself & I - G-Eazy, Bebe Rexha complete
Take Me To Church - Matt McAndrew complete
Location - Khalid complete
Omen - Disclosure, Sam Smith complete
My Boo - Ghost Town DJ's complete
Love King - The-Dream complete
Moving To Mars - Coldplay complete
The Crow & The Butterfly - Shinedown complete
Roman Reloaded - Nicki M

Losers - The Weeknd, Labrinth complete
Ride - twenty one pilots complete
If I Told You - Darius Rucker complete
A Little More Summertime - Jason Aldean complete
Heartbreak Warfare - John Mayer complete
Black Magic - Little Mix complete
The Heart Of Dixie - Danielle Bradbery complete
Conscience - Kodak Black, Future complete
Letting Go (Dutty Love) - Sean Kingston, Nicki Minaj complete
Love Me Like You Mean It - Kelsea Ballerini complete
All About Tonight - Blake Shelton complete
OTW - Khalid, Ty Dolla $ign, 6LACK complete
Somewhere Else - Toby Keith complete
Wicked - Future complete
Give Me Back My Hometown - Eric Church complete
Perfect Storm - Brad Paisley complete
Make It Rain - Matt McAndrew complete
Chasing The Sun - Hilary Duff complete
out of range
Hello - Martin Solveig, Dragonette complete
I Am Beautiful - Candice Glover complete
One Day - Matisyahu complete
Smooth Criminal - Glee Cast, 2Cellos complete
Time Warp - Glee Cast complete
Photograph - Ed Sheeran complete
Love - Lan

Me Enamore - Shakira complete
Stutter - Maroon 5 complete
The Show Goes On - Lupe Fiasco complete
Brave - Sara Bareilles complete
Pride. - Kendrick Lamar complete
Somebody That I Used To Know - Glee Cast complete
Where Do Broken Hearts Go - One Direction complete
Hold On, We're Going Home - Drake, Majid Jordan complete
Tik Tok - Glee Cast complete
Body Language - Kid Ink, Usher, Tinashe complete
New Man - Ed Sheeran complete
Outlaw - 50 Cent complete
Let Me Down Easy - Billy Currington complete
Record Year - Eric Church complete
Birthday - Katy Perry complete
Girl On Fire - Alicia Keys, Nicki Minaj complete
Please Remember Me - Danielle Bradbery complete
Take A Back Road - Rodney Atkins complete
Losin Control - Russ complete
I Got Nothin' - Darius Rucker complete
Love Yourself - Justin Bieber complete
Bellas Finals: Price Tag / Don't You (Forget About Me)... - The Barden Bellas complete
Ride - SoMo complete
66 - Lil Yachty, Trippie Redd complete
Te Bote - Casper Magico, Nio Garcia, Dar

I Would Die 4 U - Prince And The Revolution complete
Booty - Jennifer Lopez, Iggy Azalea, Pitbull complete
Kisses Down Low - Kelly Rowland complete
Backroad Song - Granger Smith complete
This Ain't Nothin' - Craig Morgan complete
To The Max - DJ Khaled, Drake complete
Love Is An Open Door - Kristen Bell, Santino Fontana complete
I Got You - Jack Johnson complete
Safe And Sound - Capital Cities complete
Mary, Did You Know? - Pentatonix complete
4 AM - 2 Chainz, Travis Scott complete
Swish Swish - Katy Perry, Nicki Minaj complete
Don't Ya - Brett Eldredge complete
Do You Want To Build A Snowman? - Kristen Bell, Agatha Lee Monn, Katie Lopez complete
Break Free - Ariana Grande, Zedd complete
Numb - Usher complete
Closer - The Chainsmokers, Halsey complete
I Just Had Sex - The Lonely Island, Akon complete
Setting Fires - The Chainsmokers, XYLO complete
Check It Out - will.i.am, Nicki Minaj complete
Dynamite - Taio Cruz complete
One More Drinkin' Song - Jerrod Niemann complete
A Year Without

Man Of The Woods - Justin Timberlake complete
Trailerhood - Toby Keith complete
There's Nothing Holdin' Me Back - Shawn Mendes complete
On My Mind - Ellie Goulding complete
Focus - H.E.R. complete
Feelin' Myself - will.i.am, Miley Cyrus, French Montana, Wiz Khalifa, DJ Mustard complete
UnFazed - Lil Uzi Vert, The Weeknd complete
The Boys of Fall - Kenny Chesney complete
Believe Me - Lil Wayne, Drake complete
Steady Mobbin' - Young Money, Gucci Mane complete
Holy Key - DJ Khaled, Big Sean, Kendrick Lamar, Betty Wright complete
Japan - Famous Dex complete
Just Breathe - Pearl Jam complete
Jimmy Choo - Fetty Wap complete
Barcelona - Ed Sheeran complete
Once An Addict (Interlude) - J. Cole complete
Both Of Us - B.o.B, Taylor Swift complete
Crazy Girl - Eli Young Band complete
Hurry Home - Jason Michael Carroll complete
Crooked Smile - J. Cole, TLC complete
Comfortable - K Camp complete
Nights - Frank Ocean complete
For You, And Your Denial - Yellowcard complete
For Good - Glee Cast complet

...

Digital Dash - Drake, Future complete
No Church In The Wild - Jay Z Kanye West, Frank Ocean complete
Love Runs Out - OneRepublic complete
Blue Notes - Meek Mill complete
Drop The World - Lil Wayne, Eminem complete
Home Alone Tonight - Luke Bryan, Karen Fairchild complete
Believer - Imagine Dragons complete
John Cougar, John Deere, John 3:16 - Keith Urban complete
Law - Yo Gotti, E-40 complete
out of range
Swang - Rae Sremmurd complete
Man In The Mirror - Adam Levine, Javier Colon complete
Love Make The World Go Round - Jennifer Lopez, Lin-Manuel Miranda complete
I Know What You Did Last Summer - Shawn Mendes, Camila Cabello complete
Nasty - Bandit Gang Marco, Dro complete
God. - Kendrick Lamar complete
How Deep Is Your Love - Calvin Harris, Disciples complete
Get Low - Zedd, Liam Payne complete
Please - Sawyer Fredericks complete
2 Phones - Kevin Gates complete
Two Black Cadillacs - Carrie Underwood complete
Love You Like That - Canaan Smith complete
You're So Beautiful - Empire Cast, 

Honest - The Chainsmokers complete
Be Careful - Cardi B complete
Marry You - Bruno Mars complete
Isn't She Lovely - Glee Cast complete
Deja Vu - 3OH!3 complete
Country Boy - Aaron Lewis, George Jones, Charlie Daniels complete
Pour It Up - Rihanna complete
Come Get Her - Rae Sremmurd complete
Paper Doll - John Mayer complete
Meant To Be - Bebe Rexha, Florida Georgia Line complete
Teenage Daughters - Martina McBride complete
Fun - Pitbull, Chris Brown complete
King Of Anything - Sara Bareilles complete
Thunder - Imagine Dragons complete
One In A Million - Ne-Yo complete
Something In The Water - Carrie Underwood complete
Starships - Nicki Minaj complete
Wasted Love - Matt McAndrew complete
Energy - Drake complete
Hold My Hand - Michael Jackson Duet With Akon complete
Hangover - PSY, Snoop Dogg complete
0 To 100 / The Catch Up - Drake complete
Jealous - Nick Jonas complete
Rewind - Rascal Flatts complete
Eenie Meenie - Sean Kingston, Justin Bieber complete
Gentleman - PSY complete
Compass 

LoveHate Thing - Wale, Sam Dew complete
Fancy - Drake, T.I., Swizz Beatz complete
Where Are U Now - Skrillex, Diplo With Justin Bieber complete
Someone Else Calling You Baby - Luke Bryan complete
Knee Deep - Zac Brown Band, Jimmy Buffett complete
Digits - Young Thug complete
Gotta Have It - Jay Z Kanye West complete
Deuces - Chris Brown, Tyga, Kevin McCall complete
Imitadora - Romeo Santos complete
If You Only Knew - Shinedown complete
Am I The Only One - Dierks Bentley complete
Naked - Dev, Enrique Iglesias complete
It's A Man's, Man's, Man's World - Juliet Simms complete
Hello - Adele complete
Strangers - Halsey, Lauren Jauregui complete
Let Me - Zayn complete
Strip That Down - Liam Payne, Quavo complete
First Love - Jennifer Lopez complete
Gas Pedal - Sage The Gemini, IamSu! complete
6 God - Drake complete
No Such Thing As A Broken Heart - Old Dominion complete
Somebody To Love - Justin Bieber, Usher complete
Mama's Song - Carrie Underwood complete
Night Train - Jason Aldean complet

Mean To Me - Brett Eldredge complete
Good Girls - 5 Seconds Of Summer complete
Beautiful Monster - Ne-Yo complete
Take Back The Night - Justin Timberlake complete
White Liar - Miranda Lambert complete
What The Hell - Avril Lavigne complete
On My Level - Wiz Khalifa, Too $hort complete
Shower - Becky G complete
Turn On The Lights - Future complete
Fireflies - Owl City complete
Gold - Britt Nicole complete
Hallelujah (Vancouver Winter 2010 Version) - k.d. lang complete
Grammys - Drake, Future complete
Friends - J. Cole, kiLL edward complete
Where I Come From - Montgomery Gentry complete
Royals - Taylor John Williams complete
Peter Pan - Kelsea Ballerini complete
679 - Fetty Wap, Remy Boyz complete
Hymn For The Weekend - Coldplay complete
Mercy - Dave Matthews Band complete
Dibs - Kelsea Ballerini complete
Radioactive - Kings Of Leon complete
I Walk The Line - Craig Wayne Boyd complete
Sexy Chick - David Guetta, Akon complete
I Invented Sex - Trey Songz, Drake complete
New Rules - Dua Lip

Diamond Teeth Samurai - YoungBoy Never Broke Again complete
Stupid Boy - Cassadee Pope complete
Live A Little - Kenny Chesney complete
The Night We Met - Lord Huron complete
Chill Bill - Rob $tone, J. Davi$, Spooks complete
Marry Me - Jason Derulo complete
Talkin' 2 Myself - Eminem, Kobe complete
Part Of Me - Katy Perry complete
F**k Love - XXXTENTACION, Trippie Redd complete
Everybody Hates Me - The Chainsmokers complete
Go Get It - T.I. complete
out of range
One Love (People Get Ready) - Glee Cast complete
You And Your Friends - Wiz Khalifa, Snoop Dogg, Ty Dolla $ign complete
Even If It Breaks Your Heart - Eli Young Band complete
Galway Girl - Ed Sheeran complete
She's Got A Way With Words - Blake Shelton complete
Depression & Obsession - XXXTENTACION complete
Sweeter Than Fiction - Taylor Swift complete
Everybody Talks - Neon Trees complete
Intro - J. Cole complete
Supastars - Migos complete
Better With The Lights Off - New Boyz, Chris Brown complete
Cat Daddy - Rej3ctz complete
Sho

In [33]:
len(uris)

3560

## 6. Break the list of uris out into chunks, and pull song information for said uri. Breaking into chunks allows for faster processing and doesn't tax the API too badly.

In [35]:
cleaned=pd.DataFrame(uris)
tracks=cleaned[0]

In [36]:
chunks = [tracks[x:x+50] for x in range(0, len(tracks), 50)]
song_details=[]
for x in chunks:
    song_details+=sp.tracks(x)['tracks']

In [37]:
song_df=pd.DataFrame(song_details)

In [41]:
song_df.drop(columns=['available_markets','disc_number','external_urls',\
                      'external_ids','href','is_local','preview_url','track_number','type'],inplace=True)

In [42]:
album1=song_df['album'].apply(pd.Series)
album1=album1['release_date']

In [44]:
song_df['collabs']=[len(x)-1 for x in song_df['artists']] #Calculates how many other artists are on the song
song_df['artist_name']=[x[0].get('name') for x in song_df['artists']] # Saving the arist and uri for arist genre processing
song_df['artist_uri']=[x[0].get('uri') for x in song_df['artists']]
song_info=pd.concat([song_df, album1], axis=1, join_axes=[song_df.index])

## 7. Audio features exist in separate from song information. We will obtain audio features in a way similar to Step 6.

In [38]:
features=[]
for x in chunks:
    features+= sp.audio_features(x)

In [39]:
y=list(range(len(features)))
feat=list(zip(y,features))
feat=pd.DataFrame(feat)
audio=feat[1].apply(pd.Series)



### Bring together the two datafiles

In [45]:
spotify_info=pd.concat([song_info, audio], axis=1, join_axes=[song_info.index])

In [52]:
spotify_info.to_csv('billboard_5_10.csv')

## 8. Let's get some information on an artist, which we will process later

In [46]:
artistry=list(spotify_info.artist_uri.unique())

In [47]:

artist_information=[]

for x in artistry[:]:
    artist_info=sp.artist(x)
    row=({'uri': x,
          'artist':artist_info['name'],
          'genres':artist_info['genres'],
          'followers':artist_info['followers'].get('total')})
    artist_information.append(row)
    print(artist_info['name']+' appended')

Glee Cast appended
David Guetta appended
21 Savage appended
Chris Brown appended
Rebecca Black appended
OneRepublic appended
Lil Wayne appended
Lorde appended
Chance The Rapper appended
Ariana Grande appended
Panic! At The Disco appended
YG appended
Chris Janson appended
Lil Uzi Vert appended
One Direction appended
Parmalee appended
G-Eazy appended
Matt McAndrew appended
Khalid appended
Disclosure appended
Ghost Town DJ appended
Elvis Presley appended
Coldplay appended
Shinedown appended
Nicki Minaj appended
Clean Bandit appended
Kane Brown appended
Josh Kaufman appended
Trey Songz appended
Cardi B appended
Ed Sheeran appended
Madonna appended
Jordan Smith appended
Starley appended
Future appended
Louis Tomlinson appended
Curren$y appended
Kenny Chesney appended
Beyoncé appended
Lupe Fiasco appended
Sam Smith appended
Passion Pit appended
The Weeknd appended
Migos appended
Britney Spears appended
Tim McGraw appended
Taylor Swift appended
Cole Swindell appended
Shakira appended
Bruno Ma

Rico Love appended
Ellie Goulding appended
H.E.R. appended
Karaoke Pop Stars appended
Famous Dex appended
Fetty Wap appended
Eli Young Band appended
Jason Michael Carroll appended
K CAMP appended
Yellowcard appended
Five Finger Death Punch appended
Holy Grail appended
BTS appended
Nickelback appended
Mary Lambert appended
A$AP Ferg appended
Lil Dicky appended
Three Days Grace appended
Sara Ramirez appended
Luke Combs appended
Zay Hilfigerrr appended
YC Worldwide appended
Plies appended
Kevin Gates appended
Nico & Vinz appended
"Weird Al" Yankovic appended
Kid Rock appended
No Doubt appended
Michael Ray appended
Dove Cameron appended
Andy Grammer appended
K. Michelle appended
Stingray Music (Karaoke) appended
Jessie J appended
Joe Budden appended
Rich Gang appended
Bad Wolves appended
Post Malone appended
Enrique Iglesias appended
The Fray appended
Of Monsters and Men appended
Tay-K appended
Offset appended
Nelly appended
Flo Rida appended
Edens Edge appended
Duck Sauce appended
Rich Th

AWOLNATION appended
O.T. Genasis appended
Uncle Kracker appended
Party Hit Kings appended
Andres Espinosa appended
Bandit Gang Marco appended
Canaan Smith appended
DJ Esco appended
Aloe Blacc appended
Weezer appended
Juliet Simms appended
Passenger appended
Godsmack appended
MKTO appended
Andy Williams appended
Jidenna appended
Avenged Sevenfold appended
Hey Violet appended
Hadouken! appended
Soundgarden appended
Trinidad James appended
Daniel Caesar appended
Melanie Martinez appended
Brandy appended
Charice appended
Jhene Aiko appended
Soko appended
Lindsey Pavao appended
Gwen Stefani appended
Chedda Da Connect appended
Calvillo Sisters appended
Janet Jackson appended
Baauer appended
Country Karaoke Cow Boys appended
Broadway for Orlando appended
J. Dash appended
Lee DeWyze appended
Miranda Cosgrove appended
Silentó appended
Koryn Hawthorne appended
Sick Puppies appended
Aaron Lewis appended
Bebe Rexha appended
Tyler Johnson appended
Carlos Vives appended
Ultimate Party Jams appended


In [48]:
artist_info=pd.DataFrame(artist_information)
artist_info.genres=artist_info.genres.astype(str)

#Removes an artist if there is no genre associated. Usually, brand new artist/ really old artists will have blank values
artist_info=artist_info[artist_info.genres!='[]'] 

In [53]:
artist_info.to_csv('arists_5_10.csv')